**Literature review**

A statistical method for examining data points collected at regular intervals throughout time in order to identify patterns and trends is called time series analysis.

Process of time series analysis

    - collecting and cleaning data 
    - create visualizations for features
    - observing the stationarity of data
    - model selection and training
    - get predictions

The first two task has been carried out in eda-visualization notebook. I will do check stationarity and chart development in this notebook.